In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-01-18 08:29:37--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
wget: /Users/ajithj/.netrc:1: unknown token "Host"
wget: /Users/ajithj/.netrc:1: unknown token "gitlab.com"
wget: /Users/ajithj/.netrc:2: warning: ‘login’ token appears before any machine name
wget: /Users/ajithj/.netrc:3: warning: ‘password’ token appears before any machine name
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  5.14MB/s    in 0.2s    

2023-01-18 08:29:37 (5.14 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print(f'Dataset length: {len(text)}')
text[:1000]

Dataset length: 1115394


"First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citizens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we become rakes: for the gods know I\nspeak this in hunger 

In [4]:
# all unique characters in the dataset
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(f'Length of vocab: {vocab_size}')


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Length of vocab: 65


In [5]:
# create a mapping of characters to integers
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}

#encoder: take a string and return a list of integers mapped to that string
encode = lambda s: [stoi[ch] for ch in s] 

#decoder: take a list of integers and return a string
decode = lambda l: ''.join([itos[i] for i in l])

str = "Hello Transformer!"
print(encode(str))
print(decode(encode(str)))

[20, 43, 50, 50, 53, 1, 32, 56, 39, 52, 57, 44, 53, 56, 51, 43, 56, 2]
Hello Transformer!


In [6]:
# Encode the entire dataset and store it as a tensor in Torch
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [7]:
# Train and Validation data sets
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

print(f'Train data length: {len(train_data)}, Valid data length: {len(val_data)}')

Train data length: 1003854, Valid data length: 111540


In [8]:
block_size = 8
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [9]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'when input in {context} ({decode(context.tolist())}), target is {target} ({decode([target.tolist()])})')

when input in tensor([18]) (F), target is 47 (i)
when input in tensor([18, 47]) (Fi), target is 56 (r)
when input in tensor([18, 47, 56]) (Fir), target is 57 (s)
when input in tensor([18, 47, 56, 57]) (Firs), target is 58 (t)
when input in tensor([18, 47, 56, 57, 58]) (First), target is 1 ( )
when input in tensor([18, 47, 56, 57, 58,  1]) (First ), target is 15 (C)
when input in tensor([18, 47, 56, 57, 58,  1, 15]) (First C), target is 47 (i)
when input in tensor([18, 47, 56, 57, 58,  1, 15, 47]) (First Ci), target is 58 (t)


In [10]:
torch.manual_seed(1337)

batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # whhat is the maximum content length for prediction?

def get_batch(split):
    #generate a random batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    # create a block_size worth of random indexes within the data - blocksize length
    ix = torch.randint(len(data) - block_size - 1, (block_size,))
    xb = torch.stack([data[i:i+block_size] for i in ix])
    yb = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return xb,yb

xb, yb = get_batch('train')
print(f'inputs: {xb.shape}, targets: {yb.shape}')
print('xb = ', xb)
print('yb = ', yb)
print('------')
for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f'when input in {context} ({decode(context.tolist())}), target is {target} ({decode([target.tolist()])})')

inputs: torch.Size([8, 8]), targets: torch.Size([8, 8])
xb =  tensor([[53, 59,  6,  1, 58, 56, 47, 40],
        [49, 43, 43, 54,  1, 47, 58,  1],
        [13, 52, 45, 43, 50, 53,  8,  0],
        [ 1, 39,  1, 46, 53, 59, 57, 43],
        [57,  0, 58, 46, 39, 52,  1, 50],
        [26, 19,  1, 20, 17, 26, 30, 37],
        [43,  8,  0,  0, 31, 43, 41, 53],
        [54, 54, 63,  1, 40, 56, 43, 43]])
yb =  tensor([[59,  6,  1, 58, 56, 47, 40, 59],
        [43, 43, 54,  1, 47, 58,  1, 58],
        [52, 45, 43, 50, 53,  8,  0, 26],
        [39,  1, 46, 53, 59, 57, 43,  0],
        [ 0, 58, 46, 39, 52,  1, 50, 43],
        [19,  1, 20, 17, 26, 30, 37,  1],
        [ 8,  0,  0, 31, 43, 41, 53, 52],
        [54, 63,  1, 40, 56, 43, 43, 42]])
------
when input in tensor([53]) (o), target is 59 (u)
when input in tensor([53, 59]) (ou), target is 6 (,)
when input in tensor([53, 59,  6]) (ou,), target is 1 ( )
when input in tensor([53, 59,  6,  1]) (ou, ), target is 58 (t)
when input in tensor([53, 5

In [11]:
# Input to the transformer
print(xb)

tensor([[53, 59,  6,  1, 58, 56, 47, 40],
        [49, 43, 43, 54,  1, 47, 58,  1],
        [13, 52, 45, 43, 50, 53,  8,  0],
        [ 1, 39,  1, 46, 53, 59, 57, 43],
        [57,  0, 58, 46, 39, 52,  1, 50],
        [26, 19,  1, 20, 17, 26, 30, 37],
        [43,  8,  0,  0, 31, 43, 41, 53],
        [54, 54, 63,  1, 40, 56, 43, 43]])


In [12]:
# Implement a Bigram LM using Torch

import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        # A simple lookup table that stores embeddings of a fixed dictionary and size.
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
        # ifx and targets are both (B, T) tensor of integers
        logits = self.token_embedding_table(idx) #(B, T, C)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens=100):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get predictions
            logits, loss = self(idx)
            
            # focus only on the last time step
            logits = logits[:, -1, :] # shape is now (B, C)
            
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B,C)
            
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)
            
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [13]:
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(f'Loss = {loss} - based on vocab_size {vocab_size}, expected loss is $ln(1/vocab_size)$ {torch.log(torch.tensor(1/vocab_size))}')

print('Generated Text:\n')
print(decode(m.generate(idx = torch.zeros((1,1), dtype=torch.long))[0].tolist()))

torch.Size([64, 65])
Loss = 4.823126792907715 - based on vocab_size 65, expected loss is $ln(1/vocab_size)$ -4.174387454986572
Generated Text:


SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [14]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [17]:
batch_size = 32
for steps in range(10000):
    
    # sample a batch of data
    xb, yb = get_batch('train')
    
    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
print(loss.item())
    

4.750803470611572


In [18]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


phhZHotYUhricLIWkza;Z!zPyhsr.DGjsBzP!js$ztSP,cgChixj;;oAmJgCSP!ZJNulD,qbPbj
YY!BfzRjNYpWnN&vmR
xav:vr'aoTXeZ!Py$kzbkbOfSYszu3XD. -GSt3fREiBegCbjQb&ZKuMgCg
-
RlnETEluFkzP,nEr&upJ?,;wUy;aSYdeNXoT;,lgnMphiBfyyagwN!z,nsrgLzeD&Gw3SOTWhYUgGMkttdY3rhZARjHoS,iI'dPkzRySV$-w!zRoRLWW:CF$gF$Nzrdo,j?'mcr'Pl&GSbScfZbj?aUYWDYw'&Q-WKpE?hV-QOdq$E?LtcPTHoxc3flzd?SP CBk$z:3b$RKH;Nv'dgf ERIVT;AXe?UuCuaPl:Ck$Eq-A iyhSP pyNmyZXaS..SxcEE!PlvmuQTeEDq-!uLUzo'a-Pn!!$zVjF$gHEr fFVXIB-Q?olyjR'ab?!uBMKR,lS'g:CuDW:t;.iADusru


## Understanding self-attention and its mathematical underpinnings

Self-attention layers in the decoder allow each position in the decoder to attend to
all positions in the decoder up to and including that position. We need to prevent leftward
information flow (i.e. from the future) in the decoder to preserve the auto-regressive property. We implement this
inside of scaled dot-product attention by masking out (setting to −∞) all values in the input
of the softmax which correspond to illegal connections. 

In [19]:
torch.manual_seed(42)

a = torch.tril(torch.ones(5,5))
a

tensor([[1., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1.]])

In [22]:
a = a / torch.sum(a, 1, keepdim=True)
a


tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000]])

In [24]:
b = torch.randint(0,10,(5,2)).float()
c = a @ b


In [25]:
print(f'a=\n{a}\nb=\n{b}\nc=\n{c}')

a=
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000]])
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.],
        [0., 4.],
        [0., 3.]])
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333],
        [3.5000, 5.0000],
        [2.8000, 4.6000]])


In [26]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [27]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)

In [28]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

True

In [29]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

In [30]:
# Version 4: self-attention!
torch.manual_seed(1337)

B,T,C = 4,8,32 # Batch, Time, and Channels
x = torch.randn(B,T,C)

# let's build a single head self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x) # B, T, 16 (head-size)
q = query(x) # (B, T, 16)
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) --> (B, T, T)

tril = torch.tril(torch.ones(T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
out.shape


torch.Size([4, 8, 16])

**Notes:**

- Attention is a communication mechanism. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with tril, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides wei by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [31]:
k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)
wei = q @ k.transpose(-2, -1)
k.var(), q.var(), wei.var()

(tensor(1.0449), tensor(1.0700), tensor(17.4690))

In [32]:
k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)
wei = q @ k.transpose(-2, -1) * head_size ** -0.5
k.var(), q.var(), wei.var()
# See how the variance is preserved in the weights

(tensor(0.9006), tensor(1.0037), tensor(0.9957))

In [37]:
class LayerNorm1d:
    
    def __init__(self, dim, eps=1e-5):
        self.eps = eps
        self.beta = torch.zeros(dim)
        self.gamma = torch.ones(dim)
    
    def __call__(self, x):
        # forward pass
        xmean = x.mean(1, keepdim=True) # batch mean
        xvar = x.var(1, keepdim=True) # batch variance
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps)
        self.out = self.gamma * xhat + self.beta
        return self.out

        

In [38]:
torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [39]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [40]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-9.5367e-09), tensor(1.0000))